In [1]:
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Fake dataset

In [2]:
X = np.random.random((100000, 100)) - 0.5
w = np.random.random((100,))
y = np.sign(X.dot(w))
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

# CatBoost

In [11]:
import catboost

train_pool = catboost.Pool(X_tr, y_tr)
test_pool = catboost.Pool(X_te, y_te)

! rm -rf catboost_model_1
clf = catboost.CatBoostClassifier(
    border_count=128,
    learning_rate=0.1,
    train_dir='catboost_model_1',
    loss_function='Logloss',
    task_type='GPU'
)

In [12]:
%%time
clf.fit(
    train_pool,
    eval_set=test_pool,
    verbose=False,
    # plot=True  # doesn't work anyway
)

CPU times: user 9.06 s, sys: 3.16 s, total: 12.2 s
Wall time: 9.75 s


In [ ]:
# visualizer doesn't work
# w = catboost.MetricVisualizer('catboost_model_1')
# w.start()

In [13]:
print roc_auc_score(y_te, clf.predict_proba(X_te)[:, 1])

0.9949300086177104


# LightGBM

In [14]:
import lightgbm

In [15]:
clf = lightgbm.LGBMModel(n_estimators=1000, objective='binary', device="gpu")

In [16]:
%%time
clf.fit(X_tr, y_tr)

CPU times: user 2min 40s, sys: 6.58 s, total: 2min 47s
Wall time: 30.2 s


LGBMModel(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
     device='gpu', learning_rate=0.1, max_depth=-1, min_child_samples=20,
     min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
     n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
     reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
     subsample_for_bin=200000, subsample_freq=1)

In [17]:
print roc_auc_score(y_te, clf.predict(X_te))

0.9883790167249863
